In [1]:
# REMOVE ME
import NotebookLoader
import matplotlib.pyplot as plt
# --------------------

import admin
from Controller import MapleController, press_and_release, sleep, locate

importing Jupyter notebook from Controller.ipynb


In [2]:
print("Created by tomergt45 (Tomergt89@gmail.com or github.com/tomergt45) ")

if not admin.isUserAdmin():
        print("Note: You must run this application as administrator")
        admin.runAsAdmin()
        
print("Note: You must play on windowed 800x600 (4:3)")
print("Note: You must change your windows UAC settings to 'Never Notify' (don't forget to turn it back after using the bot!)")

Created by tomergt45 (Tomergt89@gmail.com or github.com/tomergt45) 
Note: You must play on windowed 800x600 (4:3)
Note: You must change your windows UAC settings to 'Never Notify' (don't forget to turn it back after using the bot!)


In [3]:
controller = MapleController('settings.json')
release = None
controller.restart_cooldown('change_channel')
controller.restart_cooldown('crash_check')

In [4]:
def login_bot():
    close_news = select_character = select_world = False
    
    if controller.get_status() != 1:
        print('Maple is not running, launching now! (Note: maximum launch wait time of 3 minutes)')
        controller.launch_maple()
        
    controller.restart_cooldown('wait_for_ingame_view')
    while not controller.check_cooldown('wait_for_ingame_view', 180):
            if controller.get_status() == 1:
                try: 
                    controller.grab_frame()
                except: 
                    print('faild grabbing frame')

                if controller.check_black_screen(x=0, y=0):
                    print('black screen')
                    sleep(2) # Maple probably still loading so wait
                    
                elif controller.check_world_menu_open() and not select_world:
                    print('Successfully launched maplestory (world menu detected). Attempting to enter character...')
                    select_world = True
                    controller.grab_frame()
                    controller.select_world(controller.world)
                    sleep(4)

                elif controller.check_characters_menu_open() and not select_character:
                    print('Successfully entered characters menu, attempting to enter PIC')
                    select_character = True
                    press_and_release('enter')
                    controller.grab_frame()
                    controller.enter_PIC(controller.PIC)
                    sleep(3)

                elif controller.check_news_window_open() and not close_news:
                    sleep(1)
                    print('Annoying news window detected, closing!')
                    close_news = True
                    controller.grab_frame()
                    press_and_release('esc')
                    sleep(2)

                elif controller.check_cashshop_button_inview():
                    print('Successfully in-game! activating login skills...')
                    controller.use_login_skills()
                    return True
    return False

In [5]:
def afk_bot():
    global release
    if controller.hold_up_state:
        controller.press_up()
    
    # Check that maplestory is running and that the cashshop button is in view (indicating that you are in game)
    while controller.get_status() == 1:
        try: # Sanity check: Failing to grab frame means app is not running properly
            controller.grab_frame()

            if controller.check_unable_change_channel_dialog_open():
                press_and_release('enter')

            controller.grab_frame()
            controller.check_health_and_heal()

            if controller.hp_potion_key != controller.mana_potion_key:
                controller.grab_frame()

            if controller.check_mana():
                controller.fill_mana()
                sleep(.15)

            if controller.check_cooldown("change_direction", controller.direction_period) or not release:
                controller.restart_cooldown("change_direction")
                if release:
                    release()
                if release == controller.release_right:
                    press = controller.press_left
                    release = controller.release_left
                    press()
                else:
                    press = controller.press_right
                    release = controller.release_right
                    press()

            if controller.check_cooldown("pet_food", controller.pet_food_period):
                controller.feed_pet()
                controller.restart_cooldown('pet_food')

            controller.grab_frame()
            controller.check_health_and_heal()

            if controller.check_rune_message_open() or controller.check_cooldown('change_channel', controller.channel_period):
                release()
                controller.move_channel()
                sleep(2)
                controller.restart_cooldown('change_channel')
                press()

            controller.grab_frame()
            controller.check_health_and_heal()

            if controller.check_cooldown('buffs', controller.buffs_period):
                controller.attack()
                controller.use_buffs()
                controller.restart_cooldown('buffs')

            # REMOVE ME
            if controller.check_cooldown('period hp', 30):
                controller.grab_frame()
                controller.heal()
                controller.restart_cooldown('period hp')
                
            if controller.check_cooldown('crash_check', 60 * 60):
                controller.restart_cooldown('crash_check')
                os.system('cls')
                break

            controller.jump_or_doublejump()
            controller.check_health_and_heal()
            controller.use_uncooldowned_skills()
            controller.check_health_and_heal()
            controller.attack()
            sleep(.25)
        except Exception as e:
            print('AFK bot crashed, restarting process')
            print(e)
            break

    # Release keys left pressed
    if controller.hold_up_state:
        controller.release_up()
    if release:
        release()

In [6]:
while 1:
    if controller.get_status() == 2: # Check if maple crashed
        if controller.attempt_restart_after_crash_state: # Close crashed maple
            print('Maplestory has crashed... closing program')
            controller.close_maple()
        else: 
            print('Maplestory has crashed, exiting program... (If you want to auto restart maple in future crashes, set "attempt_restart_after_crash=true" in the settings.json file)')
            break

    # launch the game, select world, enter PIC and coninue fighting
    status = login_bot()
    if status:
        print('Starting AFK bot')
        afk_bot()
    else:
        raise Exception('Maplestory attempt to launch and login has faild after 3 minutes of wait time (this includes attempting to launch the game, select a world, channel, character and enter PIC).')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\tomer\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-85db6abc19b8>", line 11, in <module>
    status = login_bot()
  File "<ipython-input-4-dacd67871a17>", line 16, in login_bot
    if controller.check_black_screen(x=0, y=0):
  File "<string>", line 166, in check_black_screen
  File "<string>", line 75, in grab_frame
  File "<string>", line 19, in grab_maple
  File "c:\users\tomer\appdata\local\programs\python\python39\lib\site-packages\pyscreeze\__init__.py", line 135, in wrapper
    return wrappedFunction(*args, **kwargs)
  File "c:\users\tomer\appdata\local\programs\python\python39\lib\site-packages\pyscreeze\__init__.py", line 427, in _screenshot_win32
    im = ImageGrab.grab()
  File "c:\users\tomer\appdata\local\programs\python\python39\lib\site-packages\PIL\ImageGrab.py", line 46, in

TypeError: object of type 'NoneType' has no len()

True